In [ ]:
#| default_exp statistics

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import scipy.sparse as sp, re, xclib.data.data_utils as du, numpy as np, pandas as pd, os
from IPython.display import display
from torch.utils.data import Dataset
from termcolor import colored, COLORS

from sugar.core import *
from xcai.data import *

## `UpdatedDataset`

In [ ]:
#| export
METADATA_CODE = {'category': 'cat', 'see_also': 'sal', 'hyper_link': 'hlk', 'videos': 'vid', 'images': 'img', 
                 'entity': 'ent', 'canonical': 'can', 'entity_canonical_category': 'ecc', 'entity_canonical': 'enc'}

In [ ]:
#| export
class UpdatedDataset:

    @staticmethod
    def load_data_info(data_dir, type):
        ids, txt = load_raw_txt(f'{data_dir}/raw_data/{type}.raw.txt', encoding='latin-1')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_lbl_info(data_dir, x_prefix, y_prefix):
        ids, txt = load_raw_txt(f'{data_dir}/raw_data/label.{x_prefix}-{y_prefix}.raw.txt')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_metadata_info(data_dir, metadata_type, x_prefix, y_prefix, z_prefix):
        ids, txt = load_raw_txt(f'{data_dir}/raw_data/{metadata_type}.{x_prefix}-{y_prefix}-{z_prefix}.raw.txt')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def get_trn_tst_info(data_dir):
        trn_info = UpdatedDataset.load_data_info(data_dir, 'train')
        tst_info = UpdatedDataset.load_data_info(data_dir, 'test')
        return trn_info, tst_info

    @staticmethod
    def get_labels(data_dir, x_prefix, y_prefix, old_dir=None):
        if x_prefix == 'old' and y_prefix == 'old':
            trn_mat = du.read_sparse_file(f'{old_dir}/trn_X_Y.txt')
            tst_mat = du.read_sparse_file(f'{old_dir}/tst_X_Y.txt')
        else:
            trn_mat = sp.load_npz(f'{data_dir}/trn_X_Y_{x_prefix}-{y_prefix}.npz')
            tst_mat = sp.load_npz(f'{data_dir}/tst_X_Y_{x_prefix}-{y_prefix}.npz')
        
        lbl_info = UpdatedDataset.load_lbl_info(data_dir, x_prefix=x_prefix, y_prefix=y_prefix)
        
        return trn_mat, tst_mat, lbl_info

    @staticmethod
    def get_metadata(data_dir, metadata_type, x_prefix, y_prefix, z_prefix, old_dir=None):
        if x_prefix == 'old' and y_prefix == 'old' and z_prefix == 'old':
            trn_mat = du.read_sparse_file(f'{old_dir}/{metadata_type}_trn_X_Y.txt')
            tst_mat = du.read_sparse_file(f'{old_dir}/{metadata_type}_tst_X_Y.txt')
            lbl_mat = du.read_sparse_file(f'{old_dir}/{metadata_type}_lbl_X_Y.txt')
        else:
            trn_mat = sp.load_npz(f'{data_dir}/{metadata_type}_trn_X_Y_{x_prefix}-{y_prefix}-{z_prefix}.npz')
            tst_mat = sp.load_npz(f'{data_dir}/{metadata_type}_tst_X_Y_{x_prefix}-{y_prefix}-{z_prefix}.npz')
            lbl_mat = sp.load_npz(f'{data_dir}/{metadata_type}_lbl_X_Y_{x_prefix}-{y_prefix}-{z_prefix}.npz')
        
        meta_info = UpdatedDataset.load_metadata_info(data_dir, metadata_type, x_prefix, y_prefix, z_prefix)
        
        return trn_mat, tst_mat, lbl_mat, meta_info

    @staticmethod
    def load_datasets(data_dir, save_dir, metadata_type, x_prefix, y_prefix, z_prefix):
        trn_info, tst_info = UpdatedDataset.get_trn_tst_info(data_dir)
        trn_mat, tst_mat, lbl_info = UpdatedDataset.get_labels(save_dir, x_prefix, y_prefix, data_dir)
    
        main_trn_dset = MainXCDataset(trn_info, trn_mat, lbl_info)
        main_tst_dset = MainXCDataset(tst_info, tst_mat, lbl_info)
    
        trn_meta_mat, tst_meta_mat, lbl_meta_mat, meta_info = UpdatedDataset.get_metadata(save_dir, metadata_type, x_prefix, y_prefix, z_prefix, data_dir)
    
        trn_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], trn_meta_mat, lbl_meta_mat, meta_info)
        tst_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], tst_meta_mat, lbl_meta_mat, meta_info)
    
        trn_dset = XCDataset(main_trn_dset, cat_meta=trn_meta_dset)
        tst_dset = XCDataset(main_tst_dset, cat_meta=tst_meta_dset)
    
        return trn_dset, tst_dset
        

## `Dataset`

In [ ]:
#| export
class Dataset:

    @staticmethod
    def load_data_info(data_dir, type):
        ids, txt = load_raw_txt(f'{data_dir}/raw_data/{type}.raw.txt', encoding='latin-1')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_lbl_info(data_dir):
        ids, txt = load_raw_txt(f'{data_dir}/raw_data/label.raw.txt', encoding='latin-1')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def load_metadata_info(data_dir, metadata_type, suffix=''):
        if len(suffix): suffix = f'.{suffix}'
        ids, txt = load_raw_txt(f'{data_dir}/raw_data/{metadata_type}{suffix}.raw.txt')
        return {'identifier':ids, 'input_text':txt}

    @staticmethod
    def get_trn_tst_info(data_dir):
        trn_info = Dataset.load_data_info(data_dir, 'train')
        tst_info = Dataset.load_data_info(data_dir, 'test')
        return trn_info, tst_info

    @staticmethod
    def get_labels(data_dir):
        trn_mat = du.read_sparse_file(f'{data_dir}/trn_X_Y.txt') if os.path.exists(f'{data_dir}/trn_X_Y.txt') else sp.load_npz(f'{data_dir}/trn_X_Y.npz')
        tst_mat = du.read_sparse_file(f'{data_dir}/tst_X_Y.txt') if os.path.exists(f'{data_dir}/tst_X_Y.txt') else sp.load_npz(f'{data_dir}/tst_X_Y.npz')
            
        lbl_info = Dataset.load_lbl_info(data_dir)
        
        return trn_mat, tst_mat, lbl_info

    @staticmethod
    def get_metadata(data_dir, metadata_type, suffix=''):
        m_suffix = f'_{suffix}' if len(suffix) else suffix
        trn_mat = sp.load_npz(f'{data_dir}/{metadata_type}_trn_X_Y{m_suffix}.npz')
        tst_mat = sp.load_npz(f'{data_dir}/{metadata_type}_tst_X_Y{m_suffix}.npz')
        lbl_mat = sp.load_npz(f'{data_dir}/{metadata_type}_lbl_X_Y{m_suffix}.npz')
        
        meta_info = Dataset.load_metadata_info(data_dir, metadata_type, suffix)
        
        return trn_mat, tst_mat, lbl_mat, meta_info

    @staticmethod
    def load_datasets(data_dir, metadata_type, suffix=''):
        trn_info, tst_info = Dataset.get_trn_tst_info(data_dir)
        trn_mat, tst_mat, lbl_info = Dataset.get_labels(data_dir)
    
        main_trn_dset = MainXCDataset(trn_info, trn_mat, lbl_info)
        main_tst_dset = MainXCDataset(tst_info, tst_mat, lbl_info)
    
        trn_meta_mat, tst_meta_mat, lbl_meta_mat, meta_info = Dataset.get_metadata(data_dir, metadata_type, suffix)
    
        trn_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], trn_meta_mat, lbl_meta_mat, meta_info)
        tst_meta_dset = MetaXCDataset(METADATA_CODE[metadata_type], tst_meta_mat, lbl_meta_mat, meta_info)
    
        trn_dset = XCDataset(main_trn_dset, cat_meta=trn_meta_dset)
        tst_dset = XCDataset(main_tst_dset, cat_meta=tst_meta_dset)
    
        return trn_dset, tst_dset
        

## Compute statistics

In [ ]:
#| export
CODE_METADATA = {'cat':'Category', 'sal':'Seealso', 'hlk':'hyperlink', 'vid':'Videos', 'img':'Images', 
                 'ent':'Entity', 'can':'Canonical', 'ecc':'Entity Canonical Category', 'enc':'Entity Canonical'}

In [ ]:
#| export
def matrix_stats(mat):
    n_dat = mat.shape[0]
    n_lbl = mat.shape[1]

    num_dat_lbl = mat.getnnz(axis=0)
    num_lbl_dat = mat.getnnz(axis=1)

    avg_dat_lbl = num_dat_lbl.mean()
    avg_lbl_dat = num_lbl_dat.mean()

    max_dat_lbl = num_dat_lbl.max()
    max_lbl_dat = num_lbl_dat.max()

    zro_dat_lbl = np.sum(num_dat_lbl == 0)
    zro_lbl_dat = np.sum(num_lbl_dat == 0)

    stats_dict = {
        f'# Entries' : n_dat,
        f'# Features': n_lbl,
        f'Avg. Entries per feature' : avg_dat_lbl,
        f'Avg. Feature per entry'   : avg_lbl_dat,
        f'Max. Entries per feature' : max_dat_lbl,
        f'Max. Feature per entry'   : max_lbl_dat,
        f'# Features without entry' : zro_dat_lbl,
        f'# Entries without feature': zro_lbl_dat,
    }
    return stats_dict
    

In [ ]:
#| export
def main_dset_stats(dset):
    return matrix_stats(dset.data_lbl)
    

In [ ]:
#| export
def meta_dset_stats(dset):
    dat_stats = matrix_stats(dset.data_meta)
    dat_stats['Dataset'] = 'Query'
    
    lbl_stats = matrix_stats(dset.lbl_meta)
    lbl_stats['Dataset'] = 'Label'

    return [dat_stats, lbl_stats]
    

In [ ]:
#| export
def dset_stats(dset):
    stats = []
    
    main_stats = main_dset_stats(dset.data)
    main_stats['Dataset'] = 'Main'
    stats.append(main_stats)

    for o in dset.meta.values():
        meta_stats = meta_dset_stats(o)
        for s in meta_stats: s['Dataset'] = f'{s["Dataset"]} {CODE_METADATA[o.prefix]} Metadata'
        stats.extend(meta_stats)

    return stats
    

In [ ]:
#| export
def trn_tst_stats(trn_dset, tst_dset):
    trn_stats = dset_stats(trn_dset)
    for o in trn_stats: o['Split'] = 'Train'

    tst_stats = dset_stats(tst_dset)
    for o in tst_stats: o['Split'] = 'Test'

    stats = trn_stats + tst_stats
    return stats
    

In [ ]:
#| export
def print_stats(stats):
    df = pd.DataFrame(stats).set_index(['Split', 'Dataset'])
    with pd.option_context('display.precision', 2):
        display(df)
    

In [ ]:
#| export
def print_dset_stats(trn_dset, tst_dset):
    stats = trn_tst_stats(trn_dset, tst_dset)
    print_stats(stats)
    

## Text

In [ ]:
#| export
class TextDataset(Dataset):
    
    def __init__(self, dset, pattern='.*_text$'):
        self.dset, self.pattern = dset, pattern
        colors = list(COLORS.keys())
        self.colors = [colors[i] for i in np.random.permutation(len(colors))]
    
    def __getitem__(self, idx):
        o = self.dset[idx]
        return {k:v for k,v in o.items() if re.match(self.pattern, k)}

    def show(self, idxs):
        for idx in idxs:
            for i,(k,v) in enumerate(self[idx].items()):
                key = colored(k, self.colors[i], attrs=["reverse", "blink"])
                value = colored(f': {v}', self.colors[i])
                print(key, value)
            print()

    def get_head_data(self, topk=10):
        return np.argsort(self.dset.data.data_lbl.getnnz(axis=1))[:-topk:-1]

    def get_tail_data(self, topk=10):
        num = self.dset.data.data_lbl.getnnz(axis=1)
        idx = np.argsort(num)
        valid = (num > 0)[idx]
        return idx[valid][:topk]
        
    def dump_txt(self, fname, idxs):
        with open(fname, 'w') as file:
            for idx in idxs:
                for i,(k,v) in enumerate(self[idx].items()):
                    file.write(f'{k}: {v}\n')
                file.write('\n')
            
    def dump_csv(self, fname, idxs):
        df = pd.DataFrame([self[idx] for idx in idxs])
        df.to_csv(fname, index=False)

    def dump(self, fname, idxs):
        if fname.endswith('.txt'): 
            self.dump_txt(fname, idxs)
        elif fname.endswith('.csv'): 
            self.dump_csv(fname, idxs)
        else: 
            raise ValueError(f'Invalid file extension: {fname}')
        
    

## Helper

In [ ]:
#| export
def show_updated_dataset(data_dir, save_dir, metadata_type, x_prefix, y_prefix, z_prefix, idxs, use_trn=True):
    trn_dset, tst_dset = UpdatedDataset.load_datasets(data_dir, save_dir, metadata_type, x_prefix, y_prefix, z_prefix)
    print_dset_stats(trn_dset, tst_dset)
    
    txt_dset = TextDataset(trn_dset if use_trn else tst_dset)
    txt_dset.show(idxs)
    
    return trn_dset, tst_dset

def show_dataset(data_dir, metadata_type, idxs, suffix='', use_trn=True):
    trn_dset, tst_dset = Dataset.load_datasets(data_dir, metadata_type, suffix=suffix)
    print_dset_stats(trn_dset, tst_dset)
    
    txt_dset = TextDataset(trn_dset if use_trn else tst_dset)
    txt_dset.show(idxs)
    
    return trn_dset, tst_dset
    

## `__main__`

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, required=True)
    parser.add_argument('--save_dir', type=str, required=True)
    parser.add_argument('--metadata_key', type=str, required=None)
    parser.add_argument('--x_prefix', type=str, required=True)
    parser.add_argument('--y_prefix', type=str, required=True)
    parser.add_argument('--z_prefix', type=str, required=True)
    return parser.parse_args()
    

In [ ]:
if __name__ == '__main__':
    args = parse_args()
    trn_dset, tst_dset = UpdatedDataset.load_datasets(args.data_dir, args.save_dir, args.metadata_type, 
                                                      args.x_prefix, args.y_prefix, args.z_prefix)
    

In [ ]:
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/(mapped)LF-WikiSeeAlsoTitles-320K/'
save_dir = '/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/'

metadata_type = 'category'
x_prefix, y_prefix, z_prefix = 'old', 'new', 'new'

In [ ]:
trn_dset, tst_dset = UpdatedDataset.load_datasets(data_dir, save_dir, metadata_type, x_prefix, y_prefix, z_prefix)

In [ ]:
print_dset_stats(trn_dset, tst_dset)

# Entries  # Features  \
Split Dataset                                          
Train Main                        693082      573503   
      Query Category Metadata     693082      872994   
      Label Category Metadata     573503      872994   
Test  Main                        177515      573503   
      Query Category Metadata     177515      872994   
      Label Category Metadata     573503      872994   

                               Avg. Entries per feature  \
Split Dataset                                             
Train Main                                         2.75   
      Query Category Metadata                      4.16   
      Label Category Metadata                      2.62   
Test  Main                                         0.89   
      Query Category Metadata                      1.03   
      Label Category Metadata                      2.62   

                               Avg. Feature per entry  \
Split Dataset                                           
Train Main                                       2.28   
      Query Category Metadata                    5.24   
      Label Category Metadata                    3.98   
Test  Main                                       2.88   
      Query Category Metadata                    5.09   
      Label Category Metadata                    3.98   

                               Max. Entries per feature  \
Split Dataset                                             
Train Main                                         2358   
      Query Category Metadata                     36700   
      Label Category Metadata                      7824   
Test  Main                                          244   
      Query Category Metadata                      6639   
      Label Category Metadata                      7824   

                               Max. Feature per entry  \
Split Dataset                                           
Train Main                                        136   
      Query Category Metadata                     245   
      Label Category Metadata                     247   
Test  Main                                        121   
      Query Category Metadata                     247   
      Label Category Metadata                     247   

                               # Features without entry  \
Split Dataset                                             
Train Main                                            0   
      Query Category Metadata                    136111   
      Label Category Metadata                    287010   
Test  Main                                       296707   
      Query Category Metadata                    526146   
      Label Category Metadata                    287010   

                               # Entries without feature  
Split Dataset                                             
Train Main                                         95940  
      Query Category Metadata                      60302  
      Label Category Metadata                     147334  
Test  Main                                         28176  
      Query Category Metadata                      13005  
      Label Category Metadata                     147334

In [ ]:
txt_dset = TextDataset(trn_dset)

In [ ]:
txt_dset.dump(f'{save_dir}/examples/random_{metadata_type}_{x_prefix}-{y_prefix}-{z_prefix}.txt', [10, 20, 30])
txt_dset.dump(f'{save_dir}/examples/random_{metadata_type}_{x_prefix}-{y_prefix}-{z_prefix}.csv', [10, 20, 30])

In [ ]:
txt_dset.show([10, 20, 30])

data_input_text : Austroasiatic languages
lbl2data_input_text : []
cat2data_input_text : ['Austroasiatic languages', 'Language families']
cat2lbl2data_input_text : []

data_input_text : Albania
lbl2data_input_text : ['Outline of Albania', 'Bibliography of Albania']
cat2data_input_text : ['Countries in Europe', 'Albania', 'Balkan countries', 'Member states of the Organisation internationale de la Francophonie', 'Countries and territories where Albanian is an official language', 'Member states of the United Nations', 'Republics', 'Member states of the Council of Europe', 'Member states of NATO', 'Member states of the Union for the Mediterranean', 'Member states of the Organisation of Islamic Cooperation', 'States and territories established in 1912']
cat2lbl2data_input_text : [['Albania', 'Outlines of countries', 'Outlines', 'Albania-related lists'], ['Albanian studies', 'Bibliographies of countries or regions']]

data_input_text : Altaic languages
lbl2data_input_text : ['Uralo-Siberian 